In [1]:
import pandas as pd
df_gene_info = pd.read_csv('../data/compoundinfo_beta.csv')
df_gene_info.head()

,pert_id,cmap_name,target,moa,canonical_smiles,inchi_key,compound_aliases
0,BRD-A08715367,L-theanine,NaN,NaN,CCNC(=O)CCC(N)C(O)=O,DATAGRPVKZEWHA-UHFFFAOYSA-N,l-theanine
1,BRD-A12237696,L-citrulline,NaN,NaN,NC(CCCNC(N)=O)C(O)=O,RHGKLRLOHDJJDR-UHFFFAOYSA-N,l-citrulline
2,BRD-A18795974,BRD-A18795974,NaN,NaN,CCCN(CCC)C1CCc2ccc(O)cc2C1,BLYMJBIZMIGWFK-UHFFFAOYSA-N,7-hydroxy-DPAT
3,BRD-A27924917,BRD-A27924917,NaN,NaN,NCC(O)(CS(O)(=O)=O)c1ccc(Cl)cc1,WBSMZVIMANOCNX-UHFFFAOYSA-N,2-hydroxysaclofen
4,BRD-A35931254,BRD-A35931254,NaN,NaN,CN1CCc2cccc-3c2C1Cc1ccc(O)c(O)c-31,VMWNQDUVQKEIOC-UHFFFAOYSA-N,r(-)-apomorphine


In [24]:
smiles = ['CCNC(=O)CCC(N)C(O)=O', 'NC(CCCNC(N)=O)C(O)=O']

pert_id = (df_gene_info.loc[df_gene_info['canonical_smiles'] == smiles[0], 'pert_id']).values[0]
pert_id

'BRD-A08715367'

In [28]:
def smiles2pertid(df, smiles_col, pertid_col):
    pert_ids = []
    SMILES = df[smiles_col]
    for smiles in SMILES:
        pert_id = (df.loc[df[smiles_col] == smiles, pertid_col]).values[0]
        pert_ids.append(pert_id)
    return pert_ids

In [29]:
smiles2pertid(df_gene_info, 'canonical_smiles', 'pert_id')

['BRD-A08715367',
 'BRD-A12237696',
 'BRD-A18795974',
 'BRD-A27924917',
 'BRD-A35931254',
 'BRD-A39230911',
 'BRD-A77577770',
 'BRD-A86415025',
 'BRD-K05674516',
 'BRD-K10673031']

In [1]:
def check_substring(df_query, df_substring, query_col_name, substring_col_name):
    df_query['substring_present'] = df_query[query_col_name].apply(lambda x: any(sub in str(x) for sub in df_substring[substring_col_name]))
    df_query = df_query[df_query['substring_present'] == 'True']
    return df_query

In [2]:
check_substring(df_l1000_cp, df_merged, '0', 'cmap_name')

NameError: name 'df_l1000_cp' is not defined

In [1]:
query = 'TSAI002_NPC-8_XH_I22_wortmannin_10uM up'
substring = 'wortmannin'

query.__contains__(substring)

True

In [71]:
df = pd.read_csv("../data/BindingDB_IC50_updated.csv")
# print(df.columns)

smiles = list(set(df['std_smiles']))
single_smiles = smiles[0]
print(single_smiles)

# smiles_dict = {i: smiles[i] for i in range(len(smiles))}
# with open('../data/smiles_dict.txt', 'w') as file:
#     file.write(json.dumps(smiles_dict))

def xlength(y):
    return reduce(lambda sum, element: sum + 1, y, 0)

def get_zinc_tokenizer(cfg):
    long_tokens = [a for a in list(cfg._lexical_index.keys()) if xlength(a) > 1] ####
    replacements = ['$','%','^'] # ,'&']
    assert xlength(long_tokens) == len(replacements) ####xzw
    for token in replacements: 
        assert token not in cfg._lexical_index ####
    
    def tokenize(smiles):
        for i, token in enumerate(long_tokens):
            smiles = smiles.replace(token, replacements[i])
        tokens = []
        for token in smiles:
            try:
                ix = replacements.index(token)
                tokens.append(long_tokens[ix])
            except:
                tokens.append(token)
        return tokens
    
    return tokenize

CC(C[N+](C)(C)C)OC(N)=O


In [72]:
single_smiles

'CC(C[N+](C)(C)C)OC(N)=O'

In [73]:
_tokenize = get_zinc_tokenizer(zinc_grammar.GCFG)
_parser = nltk.ChartParser(zinc_grammar.GCFG)
_productions = zinc_grammar.GCFG.productions()
_prod_map = {}
for ix, prod in enumerate(_productions):
    _prod_map[prod] = ix
MAX_LEN = 277
_n_chars = len(_productions)

""" Encode a list of smiles strings into the latent space """
# assert type(single_smiles) == list
token = map(_tokenize, single_smiles)

In [74]:
tokens = []

for t in token:
    tokens.append(t[0])

In [75]:
tokens

['C',
 'C',
 '(',
 'C',
 '[',
 'N',
 '+',
 ']',
 '(',
 'C',
 ')',
 '(',
 'C',
 ')',
 'C',
 ')',
 'O',
 'C',
 '(',
 'N',
 ')',
 '=',
 'O']

In [76]:
try:
        tp = next(_parser.parse(tokens))
except:
        print("Parse tree error at")

In [77]:
productions_seq = tp.productions()
idx = np.array([_prod_map[prod] for prod in productions_seq], dtype=int)

In [78]:
one_hot = np.zeros((MAX_LEN, _n_chars), dtype=np.float32)

In [79]:
num_productions = len(idx)
if num_productions > MAX_LEN:
        print("Too large molecules, out of range")
    #print("i=  {%d} len(indices)=  {%d} num_productions = %d " % (i,len(indices),num_productions))
        one_hot[np.arange(MAX_LEN),idx[:MAX_LEN]] = 1.
else:    
        one_hot[np.arange(num_productions),idx] = 1.
        one_hot[np.arange(num_productions, MAX_LEN),-1] = 1.

In [81]:
one_hot

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [106]:
def smiles2gvaefeature(s):

    def xlength(y):
        return reduce(lambda sum, element: sum + 1, y, 0)

    def get_zinc_tokenizer(cfg):
        long_tokens = [a for a in list(cfg._lexical_index.keys()) if xlength(a) > 1] ####
        replacements = ['$','%','^'] # ,'&']
        assert xlength(long_tokens) == len(replacements) ####xzw
        for token in replacements: 
            assert token not in cfg._lexical_index ####
        
        def tokenize(smiles):
            for i, token in enumerate(long_tokens):
                smiles = smiles.replace(token, replacements[i])
            tokens = []
            for token in smiles:
                try:
                    ix = replacements.index(token)
                    tokens.append(long_tokens[ix])
                except:
                    tokens.append(token)
            return tokens
        
        return tokenize
    
    _tokenize = get_zinc_tokenizer(zinc_grammar.GCFG)
    _parser = nltk.ChartParser(zinc_grammar.GCFG)
    _productions = zinc_grammar.GCFG.productions()
    _prod_map = {}
    for ix, prod in enumerate(_productions):
        _prod_map[prod] = ix
    MAX_LEN = 277
    _n_chars = len(_productions)
    one_hot = np.zeros((MAX_LEN, _n_chars), dtype=np.float32)

    token = map(_tokenize, s)
    tokens = []
    for t in token:
        tokens.append(t[0])
    tp = None
    try:
        tp = next(_parser.parse(tokens))
    except:
        print("Parse tree error")
        return one_hot

    productions_seq = tp.productions()
    idx = np.array([_prod_map[prod] for prod in productions_seq], dtype=int)
    num_productions = len(idx)
    if num_productions > MAX_LEN:
            print("Too large molecules, out of range")
            one_hot[np.arange(MAX_LEN),idx[:MAX_LEN]] = 1.
    else:    
            one_hot[np.arange(num_productions),idx] = 1.
            one_hot[np.arange(num_productions, MAX_LEN),-1] = 1.

    return one_hot

In [61]:
smiles

['CC(C[N+](C)(C)C)OC(N)=O',
 'Cc1ccc2cc3c(ccc4ccccc43)c3c2c1CC3',
 'CC1(C)[C@@H](OC(=O)CCC(=O)O)CC[C@]2(C)[C@H]3C(=O)C=C4[C@@H]5C[C@@](C)(C(=O)O)CC[C@]5(C)CC[C@@]4(C)[C@]3(C)CC[C@@H]12',
 'Cc1nc2n(c(=O)c1CCN1CCC(c3noc4cc(F)ccc34)CC1)CCCC2',
 'C[N+](C)(C)CCOC(N)=O',
 'Cc1sc2c(c1C)C(c1ccc(Cl)cc1)=N[C@@H](CC(=O)OC(C)(C)C)c1nnc(C)n1-2',
 'Oc1nc(SCc2ccc(Cl)cc2)nc2c1CCC2',
 'NC(=O)c1noc([C@H](CCCC2CCCCC2)CC(=O)NO)n1',
 'O=c1c(O)c(-c2ccc(O)cc2)oc2cc(O)cc(O)c12',
 'COc1cccc(-c2ccc3ncnc(NCc4cc(C)cs4)c3c2)c1',
 'O=c1c(O)c(-c2ccc(O)cc2O)oc2cc(O)cc(O)c12',
 'O=c1c2cc(F)ccc2sn1-c1ccccc1Cl',
 'CN1CCN(c2ccc3nc(-c4c(N)c5c(F)cccc5[nH]c4=O)[nH]c3c2)CC1',
 'COc1cc2ncn(-c3cc(OCc4ccccc4C(F)(F)F)c(C(N)=O)s3)c2cc1OC',
 'COCc1cc(O)c(O)c(O)c1-c1c(COC)cc(O)c(O)c1O',
 'CC[C@H](Nc1ncnc2[nH]cnc12)c1nc2cccc(F)c2c(=O)n1-c1ccccc1',
 'C[C@@H]1CN(Cc2ccc(F)cc2)[C@@H](C)CN1C(=O)c1cc2c(C(=O)C(=O)N(C)C)cn(C)c2cc1Cl',
 'CCCN1CCc2cccc3c2[C@H]1Cc1ccc(O)c(O)c1-3',
 'Cc1c(OCC(F)(F)F)ccnc1CS(=O)c1nc2ccccc2[nH]1',
 'CC1(C)NC(=O)N

In [109]:
vec = []
for i in range(len(smiles[:1000])):
    x = smiles2gvaefeature(smiles[i])
    vec.append(x)

Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree error
Parse tree err

In [111]:
len(vec)

1000